# Practice IV - Sentiment Analysis
## 1ra Parte Corpus preprocessing

In [109]:
import re, unicodedata, pandas as pd, os, spacy
from spacy.lang.es.stop_words import STOP_WORDS as SPACY_STOP_ES

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

Cargar Rest_Mex_2022

In [110]:
# Cargar corpus .xlsx
df = pd.read_excel('Recursos Profe/Rest_Mex_2022.xlsx', engine='openpyxl')
df.head(10)

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel
5,El peor huevos Benedict jamás,Hoy teníamos el desayuno por segunda vez en po...,1,Restaurant
6,Definitivamente no volvería a hospedarme en Sa...,"El hotel en si no es malo, pero mi experiencia...",1,Hotel
7,Terrible,No estoy seguro de por qué este restaurante ti...,1,Restaurant
8,"Bebidas ADULTERADAS, FATAL mi experiencia!!!",Llegué a este hotel “por desgracia” mi reserva...,1,Hotel
9,Hotel ha ido cuesta abajo,Hemos estado viniendo a Villa la Estancia dura...,1,Hotel


In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30212 entries, 0 to 30211
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       30210 non-null  object
 1   Opinion     30210 non-null  object
 2   Polarity    30212 non-null  int64 
 3   Attraction  30212 non-null  object
dtypes: int64(1), object(3)
memory usage: 944.3+ KB


In [112]:
df.shape

(30212, 4)

Concatenar Title + Opinion como texto de entrada.

In [113]:
# Convertir las columnas 'Title' y 'Opinion' a tipo string.
df['Title'] = df['Title'].astype(str)
df['Opinion'] = df['Opinion'].astype(str)

# Concatenar Title + Opinion como texto de entrada.
df['text'] = df['Title'] + ' ' + df['Opinion']

Dejar Polarity como etiqueta.

In [114]:
df['target'] = df['Polarity']

Normalización base:
- Minúsculas, quitar signos raros, espacios extra, etc.
- Manejo básico de stopwords.
- Manejo de negaciones.
- Manejo de repeticiones.
- Corrección básica de errores comunes.

Se añade:
- Emojis lexicon
- SEL_full

In [115]:
# Crear diccionario de traducción inglés -> español para términos comunes
en2es = {
    "front desk": "recepcion",
    "check-in": "check_in",
    "check in": "check_in",
    "early check-in": "check_in_temprano",
    "late check-out": "check_out_tarde",
    "check-out": "check_out",
    "check out": "check_out",
    "housekeeping": "limpieza_cuarto",
    "room service": "servicio_cuarto",
    "customer service": "servicio_cliente",
    "concierge": "conserje",
    "bellboy": "botones",
    "valet parking": "valet_parking",
    "self parking": "estacionamiento_autoservicio",
    "parking lot": "estacionamiento",
    "booking": "reserva",
    "overbooking": "sobre_reserva",
    "reservation": "reserva",
    "upgrade": "mejora_habitacion",
    "downgrade": "degradacion_habitacion",
    "no-show": "no_show",
    "walk-in": "llegada_sin_reserva",
    "timeshare": "tiempos_compartidos",
    "all inclusive": "todo_incluido",
    "half board": "media_pension",
    "full board": "pension_completa",
    "resort fee": "cargo_resort",
    "resort": "resort",
    "security deposit": "deposito_garantia",
    "refund": "reembolso",
    "chargeback": "contracargo",

    # --- Habitación / instalaciones ---
    "king bed": "cama_king",
    "queen bed": "cama_queen",
    "twin beds": "camas_individuales",
    "double bed": "cama_matrimonial",
    "sofa bed": "sofa_cama",
    "bunk bed": "literas",
    "extra bed": "cama_extra",
    "rollaway bed": "cama_plegable",
    "crib": "cuna",
    "mattress": "colchon",
    "pillow": "almohada",
    "blanket": "cobija",
    "linens": "sabanas",
    "towels": "toallas",
    "toilet paper": "papel_higienico",
    "bathroom": "bano",
    "bathtub": "banera",
    "hot tub": "jacuzzi",
    "jacuzzi": "jacuzzi",
    "shower": "ducha",
    "rain shower": "ducha_lluvia",
    "water pressure": "presion_agua",
    "air conditioning": "aire_acondicionado",
    "heater": "calefaccion",
    "safe box": "caja_fuerte",
    "mini bar": "minibar",
    "coffee maker": "cafetera",
    "hair dryer": "secadora_cabello",
    "iron": "plancha",
    "iron board": "tabla_planchar",
    "balcony": "balcon",
    "terrace": "terraza",
    "ocean view": "vista_mar",
    "sea view": "vista_mar",
    "garden view": "vista_jardin",
    "city view": "vista_ciudad",
    "soundproof": "insonorizado",
    "elevator": "elevador",
    "lift": "elevador",
    "wheelchair": "silla_ruedas",
    "wheelchair accessible": "acceso_silla_ruedas",
    "roll-in shower": "ducha_accesible",
    "accessible room": "habitacion_accesible",

    # --- Limpieza / plagas / olores ---
    "mold": "moho",
    "mildew": "moho",
    "leak": "fuga",
    "leaking": "fuga",
    "stain": "mancha",
    "stains": "manchas",
    "dusty": "polvoso",
    "dirty": "sucio",
    "smelly": "mal_olor",
    "bad smell": "mal_olor",
    "odor": "olor",
    "bedbug": "chinche_cama",
    "bedbugs": "chinches_cama",
    "cockroach": "cucaracha",
    "cockroaches": "cucarachas",
    "ants": "hormigas",
    "bugs": "insectos",

    # --- Personal / trato ---
    "staff": "personal",
    "friendly": "amable",
    "helpful": "servicial",
    "rude": "grosero",
    "impolite": "descortes",
    "unprofessional": "poco_profesional",
    "manager": "gerente",
    "host": "anfitrion",
    "hostess": "anfitriona",
    "waiter": "mesero",
    "waitress": "mesera",
    "server": "mesero",
    "bartender": "barman",
    "lifeguard": "salvavidas",

    # --- Comida / bebida ---
    "breakfast": "desayuno",
    "buffet": "buffet",
    "brunch": "brunch",
    "lunch": "comida",
    "dinner": "cena",
    "dessert": "postre",
    "appetizer": "entrada",
    "main course": "plato_fuerte",
    "menu": "menu",
    "dish": "platillo",
    "portion": "porcion",
    "taste": "sabor",
    "tasteless": "insipido",
    "delicious": "delicioso",
    "undercooked": "crudo",
    "overcooked": "sobre_cocido",
    "cold food": "comida_fria",
    "hot food": "comida_caliente",
    "drink": "bebida",
    "drinks": "bebidas",
    "beverage": "bebida",
    "beer": "cerveza",
    "wine": "vino",
    "cocktail": "coctel",
    "water": "agua",
    "ice": "hielo",
    "inclusive drinks": "bebidas_incluidas",

    # --- Zonas / amenidades ---
    "pool": "alberca",
    "infinity pool": "alberca_infinita",
    "pool bar": "bar_alberca",
    "beach": "playa",
    "beach towel": "toalla_playa",
    "private beach": "playa_privada",
    "spa": "spa",
    "sauna": "sauna",
    "steam room": "vapor",
    "gym": "gimnasio",
    "kids club": "club_ninos",
    "playground": "area_juegos",
    "shuttle": "transporte",
    "airport shuttle": "transporte_aeropuerto",
    "uber": "uber",
    "taxi": "taxi",
    "tour": "tour",
    "excursion": "excursion",

    # --- Tecnología / internet ---
    "wifi": "wifi",
    "wi-fi": "wifi",
    "internet": "internet",
    "signal": "senal",
    "coverage": "cobertura",
    "password": "contrasena",
    "tv": "tv",
    "smart tv": "smart_tv",
    "channel": "canal",
    "channels": "canales",
    "remote control": "control_remoto",
    "outlet": "contacto",
    "power outlet": "contacto",
    "charger": "cargador",

    # --- Precios / pagos / cargos ---
    "price": "precio",
    "expensive": "caro",
    "cheap": "barato",
    "fee": "cargo",
    "charge": "cargo",
    "extra charge": "cargo_extra",
    "hidden fee": "cargo_oculto",
    "tip": "propina",
    "tips": "propinas",

    # --- Ruido / seguridad ---
    "noise": "ruido",
    "noisy": "ruidoso",
    "thin walls": "paredes_delgadas",
    "security": "seguridad",
    "safe": "seguro",
    "unsafe": "inseguro",
    "theft": "robo",
    "stolen": "robado",

    # --- Valoraciones rápidas ---
    "amazing": "increible",
    "awesome": "increible",
    "great": "genial",
    "good": "bueno",
    "ok": "regular",
    "average": "promedio",
    "bad": "malo",
    "terrible": "terrible",
    "awful": "horrible",
    "horrible": "horrible",
    "disappointing": "decepcionante",
    "worth it": "vale_la_pena",
    "not worth it": "no_vale_la_pena",
}

In [116]:
stop_es = set(SPACY_STOP_ES)
negaciones = {"no","nunca","jamás","jamas","ni","sin", "tampoco","nadie","nada",
              "ningun","ningún","ninguno","ninguna","ningunos","ningunas"}
NEG_SINGLE = {"no","nunca","jamas","jamás","tampoco","nadie","nada","ni","sin",
              "ningun","ningún","ninguno","ninguna","ningunos","ningunas"}
NEG_BIGRAMS = {("ya","no"),("ni","siquiera"),("para","nada"),("en","absoluto"),("sin","que")}
stop_es_base = stop_es - negaciones
stop_en_basic = {"the","a","an","and","or","but","is","are","was","were","be","been","to","of",
                 "in","on","at","for","with","by","about","from","it","this","that","very",
                 "really","so","just","only","my","our","your","their","we","they"}
TIME_UNITS = {"am","pm","hora","horas","minuto","minutos","segundo","segundos"}

_re_url   = re.compile(r"https?://\S+|www\.\S+", re.I)
_re_email = re.compile(r"\b[\w.+-]+@[\w-]+\.[\w.-]+\b")
_re_money = re.compile(r"(?<!\d)(\$)\s*\d+[.,]?\d*")
_re_num = re.compile(r"(?<![\w%$])\d+[.,]?\d*(?![\w%$])")
_re_space = re.compile(r"\s+")
_re_risa  = re.compile(r"j+a+(j+a+)+", re.I)
_re_repe  = re.compile(r"(.)\1{2,}", re.UNICODE)
_re_emoji = re.compile("[" "\U0001F300-\U0001FAFF" "\U00002700-\U000027BF" "\U00002600-\U000026FF" "]+")
_re_punct = re.compile(r"[^\w\s%$]")

# Carga spaCy en español, excluyendo lo que no se necesita
nlp = spacy.load("es_core_news_sm", exclude=["ner","parser","senter","textcat","tok2vec"])
BATCH, NPROC = 4000, 6

# Emoji Lexicon
emo_df  = pd.read_excel("Recursos Profe/Emojis lexicon.XLSX")
emo_df.columns = [str(c).strip().lower() for c in emo_df.columns]
emo_cols = ["anger","anticipation","disgust","fear","joy","sadness","surprise","trust"]
sent_cols = ["negative","positive"]
emoji_list = emo_df["emoji"].astype(str).tolist()
weights_emotions = {
    e: emo_df.set_index("emoji")[e].to_dict() for e in emo_cols
}
weights_sent = {
    s: emo_df.set_index("emoji")[s].to_dict() for s in sent_cols
}

def emoji_weighted_features(raw_text: str):
    # Cuenta ocurrencias por emoji (algunos pueden repetirse)
    # Nota: .count() funciona bien para emojis de un caracter; si tuvieras secuencias, usa regex.
    counts = {em: raw_text.count(em) for em in emoji_list}
    total = sum(counts.values())

    # Acumula puntajes ponderados por ocurrencia
    feat = {}
    # a) emociones básicas
    for e in emo_cols:
        s = 0.0
        wdict = weights_emotions[e]
        for em, c in counts.items():
            if c:
                w = wdict.get(em, 0.0)
                s += c * float(w)
        feat[f"emoji_{e}_score"] = s

    # b) polaridad agregada
    pos = neg = 0.0
    for em, c in counts.items():
        if c:
            pos += c * float(weights_sent["positive"].get(em, 0.0))
            neg += c * float(weights_sent["negative"].get(em, 0.0))
    feat["emoji_pos_score"] = pos
    feat["emoji_neg_score"] = neg
    feat["emoji_score"] = pos - neg

    # c) conteos crudos y densidades
    feat["emoji_count"] = int(total)
    # normaliza por longitud para que no “premie” textos largos (opcional)
    L = max(len(raw_text), 1)
    feat["emoji_density"] = total / L

    return pd.Series(feat)

# Lemtacización
def rejoin_negations_after_lemma(text_lemmatized: str):
    toks = text_lemmatized.split()
    out = []; i = 0
    while i < len(toks):
        tk = toks[i]
        # Bigramas
        if i+1 < len(toks) and (tk, toks[i+1]) in NEG_BIGRAMS:
            pair = f"{tk}_{toks[i+1]}"
            if i+2 < len(toks) and toks[i+2] not in stop_es_base:
                out.append(f"{pair}_{toks[i+2]}"); i += 3
            else:
                out.append(pair); i += 2
            continue
        # 'sin' solo
        if tk == "sin":
            if i+1 < len(toks) and toks[i+1] not in stop_es_base:
                out.append(f"sin_{toks[i+1]}"); i += 2
            else:
                out.append("sin"); i += 1
            continue
        # Un solo token (no, nunca, ni, tampoco, ningún…)
        if tk in NEG_SINGLE:
            if i+1 < len(toks):
                nxt = toks[i+1]
                if (nxt not in stop_es_base and nxt != "num" and nxt not in TIME_UNITS):
                    out.append(f"{tk}_{nxt}"); i += 2; continue
            out.append(tk); i += 1; continue
        out.append(tk); i += 1
    return " ".join(out)

# Funciones de normalización
def strip_accents(s): 
    nfkd = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in nfkd if not unicodedata.combining(ch))

def reduce_repeated(text): return _re_repe.sub(r"\1\1", text)

def replace_en_domain_words(text):
    for en, es in en2es.items():
        text = re.sub(rf"\b{re.escape(en)}\b", es, text, flags=re.I)
    return text

def normalize_text_for_lemma(s):
    s = str(s).lower().replace("’","'").replace("“",'"').replace("”",'"')
    s = _re_url.sub(" url ", s); s = _re_email.sub(" email ", s)
    s = s.replace(":)", " emoticon_positivo ").replace("(:", " emoticon_positivo ")
    s = s.replace(":(", " emoticon_negativo ").replace("):", " emoticon_negativo ")
    s = _re_emoji.sub(" emoji ", s)
    s = _re_risa.sub(" jaja ", s); s = reduce_repeated(s)
    s = _re_money.sub(" dinero ", s); s = _re_num.sub(" num ", s)
    # ⛔ aquí NO llames strip_accents
    s = _re_punct.sub(" ", s)
    s = replace_en_domain_words(s)
    s = _re_space.sub(" ", s).strip()
    toks = [t for t in s.split() if (t not in stop_es_base and t not in stop_en_basic)]
    return " ".join(toks)

# SEL
sel = pd.read_csv("Recursos Profe/SEL_full.txt", sep=r"\s*\t\s*", engine="python")
sel["key"] = sel["Palabra"].str.lower().apply(strip_accents)
pfa = dict(zip(sel["key"], sel["PFA"]))
cat = dict(zip(sel["key"], sel["Categoría"].str.lower()))
emociones = sorted(sel["Categoría"].str.lower().unique())

def sel_features(text_norm):
    toks = text_norm.split()
    score = 0.0
    counts = {e: 0 for e in emociones}
    for t in toks:
        w = t.replace("no_",""); inv = t.startswith("no_")
        if w in pfa: score += (-pfa[w] if inv else pfa[w])
        if w in cat: counts[cat[w]] += (-1 if inv else 1)
    counts["sel_score"] = score
    return pd.Series(counts)

Aplicar al dataframe

In [117]:
# Limpieza rápida
df["text_clean"] = df["text"].apply(normalize_text_for_lemma)
texts = df["text_clean"].tolist()

In [118]:
# Lematización en lote con spaCy
lemmas = []
for doc in nlp.pipe(texts, batch_size=BATCH, n_process=NPROC):
    lemmas.append(" ".join(t.lemma_ for t in doc))

lemmas = [strip_accents(t) for t in lemmas]

df["text_norm"] = [rejoin_negations_after_lemma(t) for t in lemmas]
df.drop(columns=["text_clean"], inplace=True)

In [119]:
# Emoji lexicon
df[[
    "emoji_anger_score","emoji_anticipation_score","emoji_disgust_score","emoji_fear_score",
    "emoji_joy_score","emoji_sadness_score","emoji_surprise_score","emoji_trust_score",
    "emoji_pos_score","emoji_neg_score","emoji_score","emoji_count","emoji_density"
]] = df["text"].apply(emoji_weighted_features)

In [120]:
# SEL
df[["sel_score", *emociones]] = df["text_norm"].apply(sel_features)

In [121]:
df.head()

,Title,Opinion,Polarity,Attraction,text,target,text_norm,emoji_anger_score,emoji_anticipation_score,emoji_disgust_score,...,emoji_score,emoji_count,emoji_density,sel_score,alegría,enojo,miedo,repulsión,sorpresa,tristeza
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel,Pésimo lugar Piensen dos veces antes de ir a e...,1,pesimo lugar piensen hotel molestan hijo anos ...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.630
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant,No vayas a lugar de Eddie Cuatro de nosotros f...,1,no_vayas lugar eddie recientemente eddie s pla...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.898
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel,Mala relación calidad-precio seguiré corta y s...,1,mala relacion calidad precio seguire corta sim...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.265
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel,Minusválido? ¡No te alojes aquí! Al reservar u...,1,minusvalido no_alojes reservar hotel multiprop...,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.529
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel,Es una porqueria no pierdan su tiempo No pierd...,1,porqueria no_pierdan tiempo no_pierdan tiempo ...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.794


Se exporta archivo normalizado

In [122]:
df.to_excel("Rest_Mex_2022_preprocessed.xlsx", index=False, engine='openpyxl')